In [1]:
print("Hello World")

Hello World


In [4]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain



In [6]:
%pwd

'/Users/basonpark/Desktop/genAIprojects/genai-sourcecode-analyzer/research'

In [7]:
!mkdir test_repo

In [12]:
repo_path = "test_repo/"
repo = Repo.clone_from("https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI", repo_path)



GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI test_repo/
  stderr: 'fatal: destination path 'test_repo' already exists and is not an empty directory.
'

In [13]:
loader = GenericLoader.from_filesystem(repo_path,
                                       glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON,
                                                              parser_threshold=500))

documents = loader.load()

documents

[Document(metadata={'source': 'test_repo/setup.py', 'language': <Language.PYTHON: 'python'>}, page_content="from setuptools import find_packages, setup\n\nsetup(\n    name = 'Generative AI Project',\n    version= '0.0.0',\n    author= 'Bappy Ahmed',\n    author_email= 'entbappy73@gmail.com',\n    packages= find_packages(),\n    install_requires = []\n\n)"),
 Document(metadata={'source': 'test_repo/app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()\n\nPINECONE_API_KEY=os.environ.ge

In [16]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                                  chunk_size=500,
                                                                  chunk_overlap=20)

texts = documents_splitter.split_documents(documents)

texts


[Document(metadata={'source': 'test_repo/setup.py', 'language': <Language.PYTHON: 'python'>}, page_content="from setuptools import find_packages, setup\n\nsetup(\n    name = 'Generative AI Project',\n    version= '0.0.0',\n    author= 'Bappy Ahmed',\n    author_email= 'entbappy73@gmail.com',\n    packages= find_packages(),\n    install_requires = []\n\n)"),
 Document(metadata={'source': 'test_repo/app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()'),
 Document(metadata={'source': 

In [17]:
len(texts)

13

In [19]:
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [20]:
embeddings = OpenAIEmbeddings(disallowed_special=())

/var/folders/0c/s17k17lj5ddczgmk0m7ym8nm0000gn/T/ipykernel_50674/805463212.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(disallowed_special=())


In [23]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory="./data")

In [25]:
vectordb.persist()

In [26]:
llm = ChatOpenAI()

/var/folders/0c/s17k17lj5ddczgmk0m7ym8nm0000gn/T/ipykernel_50674/44520985.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()


In [27]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

/var/folders/0c/s17k17lj5ddczgmk0m7ym8nm0000gn/T/ipykernel_50674/2101274949.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm=llm, memory=memory, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

q